<a href="https://colab.research.google.com/github/saqlainkazi690/Projects/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: import google drive'

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import os

In [3]:
import os


paired_dir = '/content/drive/MyDrive/EUVP Dataset/Paired'


def collect_image_paths(directory):
    poor_images = []
    enhanced_images = []
    validation_images = []

    for root, dirs, files in os.walk(directory):
        for file in files:

            if 'trainA' in root and file.endswith('.jpg'):
                poor_images.append(os.path.join(root, file))

            elif 'trainB' in root and file.endswith('.jpg'):
                enhanced_images.append(os.path.join(root, file))

            elif 'validation' in root and file.endswith('.jpg'):
                validation_images.append(os.path.join(root, file))

    return poor_images, enhanced_images, validation_images

dark_poor, dark_enhanced, dark_validation = collect_image_paths(os.path.join(paired_dir, 'underwater_dark'))


imagenet_poor, imagenet_enhanced, imagenet_validation = collect_image_paths(os.path.join(paired_dir, 'underwater_imagenet'))


scenes_poor, scenes_enhanced, scenes_validation = collect_image_paths(os.path.join(paired_dir, 'underwater_scenes'))

poor_images = dark_poor + imagenet_poor + scenes_poor
enhanced_images = dark_enhanced + imagenet_enhanced + scenes_enhanced
validation_list = dark_validation + imagenet_validation + scenes_validation

print("Number of poor images:", len(poor_images))
print("Number of enhanced images:", len(enhanced_images))
print("Number of validation images:", len(validation_list))
print(poor_images[0:5])
print(enhanced_images[0:6])

Number of poor images: 9020
Number of enhanced images: 9020
Number of validation images: 1732
['/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainA/269038_00006117.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainA/270258_scuba_phi_phi_791.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainA/267852_00036420.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainA/272708_00014881.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainA/272507_00000748.jpg']
['/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainB/267852_00036420.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainB/272708_00014881.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainB/270258_scuba_phi_phi_791.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainB/269038_00006117.jpg', '/content/drive/MyDrive/EUVP Dataset/Paired/underwater_dark/trainB/272507_00000748.jp

In [ ]:
# prompt: i have two lists one of poor images paths and one of enhanced images paths i need to train a conditional gan model to compress poor images and generate enhanced images after compression and finally get the mse value between the original and generated image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
poor_images_data = []
enhanced_images_data = []
for image_path in poor_images:
    image_data = tf.io.read_file(image_path)
    image_data = tf.image.decode_jpeg(image_data)
    poor_images_data.append(image_data)

for image_path in enhanced_images:
    image_data = tf.io.read_file(image_path)
    image_data = tf.image.decode_jpeg(image_data)
    enhanced_images_data.append(image_data)

# Define the generator model
generator = keras.Sequential([
  layers.Dense(256, input_shape=(100,)),
  layers.LeakyReLU(alpha=0.2),
  layers.Dense(512),
  layers.LeakyReLU(alpha=0.2),
  layers.Dense(1024),
  layers.LeakyReLU(alpha=0.2),
  layers.Dense(2048),
  layers.LeakyReLU(alpha=0.2),
  layers.Dense(4096, activation='tanh')
])

# Define the discriminator model
discriminator = keras.Sequential([
  layers.Dense(4096, input_shape=(4096,)),
  layers.LeakyReLU(alpha=0.2),
  layers.Dropout(0.3),
  layers.Dense(2048),
  layers.LeakyReLU(alpha=0.2),
  layers.Dropout(0.3),
  layers.Dense(1024),
  layers.LeakyReLU(alpha=0.2),
  layers.Dropout(0.3),
  layers.Dense(512),
  layers.LeakyReLU(alpha=0.2),
  layers.Dropout(0.3),
  layers.Dense(256),
  layers.LeakyReLU(alpha=0.2),
  layers.Dropout(0.3),
  layers.Dense(1, activation='sigmoid')
])

# Define the conditional GAN model
cgan = keras.Sequential([
  generator,
  discriminator
])

# Compile the cgan model
cgan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Train the cgan model
cgan.fit(poor_images_data, enhanced_images_data, epochs=100, batch_size=32)

# Evaluate the cgan model
mse = tf.keras.losses.MeanSquaredError()
generated_images = generator.predict(poor_images)
mse_value = mse(generated_images, enhanced_images).numpy()

print("MSE value:", mse_value)
